In [12]:
import os
import sys

print("🚀 Starting updated installation...")

# 1. Install system dependencies first (crucial for building Vina/OpenFold)
print("🛠️ Installing system tools...")
!apt-get -qq update
!apt-get -qq install -y autodock-vina swig libboost-all-dev > /dev/null 2>&1

# 2. Install core packages with flexible versions
print("📦 Installing core packages...")
!pip install -q gradio==4.19.0 py3Dmol==2.0.4 biopython==1.83 plotly==5.18.0

# 3. Install PyTorch & JAX
print("🔥 Installing PyTorch & JAX...")
!pip install -q torch torchvision
!pip install -q --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# 4. Install ESMFold & OpenFold
print("🧬 Installing Folding tools...")
!pip install -q fair-esm[esmfold]==2.0.0
!pip install -q 'dllogger @ git+https://github.com/NVIDIA/dllogger.git'
# Using a more stable install path for OpenFold dependencies
!pip install -q 'openfold @ git+https://github.com/aqlaboratory/openfold.git@4b41059694619831a7db195b7e0988fc4ff3a307'

# 5. Install ColabFold
print("🔬 Installing AlphaFold2 (ColabFold)... ")
!pip install -q "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"

# 6. Install MD tools (Fixed pdbfixer version)
print("🧪 Installing MD simulation tools...")
!pip install -q openmm pdbfixer mdtraj

# 7. Install docking tools (Fixed Vina building issue)
print("🎯 Installing docking tools...")
!pip install -q vina meeko rdkit

print("\n✅ Installation complete! Please run the import cell (the next one) to finish setup.")

🚀 Starting updated installation...
🛠️ Installing system tools...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
📦 Installing core packages...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
colabfold 1.5.5 requires numpy<3.0.0,>=2.0.2, but you have numpy 1.26.4 which is incompatible.
mdtraj 1.11.1.post1 requires numpy~=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.9.0.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.9.0.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is 

In [13]:
import os
import sys
import subprocess

def ensure_installed(package_name, import_name=None):
    if import_name is None:
        import_name = package_name
    try:
        __import__(import_name)
    except ImportError:
        print(f"📦 {import_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package_name])

# Ensure critical libraries are present
ensure_installed("py3Dmol==2.0.4", "py3Dmol")
ensure_installed("biopython==1.83", "Bio")

import gradio as gr
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
import tempfile
import warnings
import re
from Bio import SeqIO
from Bio.PDB import PDBParser, PDBIO

warnings.filterwarnings('ignore')

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 Using device: {device}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Create output directory
os.makedirs('/content/outputs', exist_ok=True)
os.makedirs('/content/temp', exist_ok=True)

print("\n✅ Libraries imported successfully!")

💻 Using device: cuda
🎮 GPU: Tesla T4
💾 Memory: 15.8 GB

✅ Libraries imported successfully!


In [14]:
# Global variables for models
esmfold_model = None
alphafold_runner = None

def load_esmfold():
    """Load ESMFold model"""
    global esmfold_model

    if esmfold_model is not None:
        return esmfold_model

    try:
        print("Loading ESMFold model (first time only, ~2-3 minutes)...")
        import esm

        esmfold_model = esm.pretrained.esmfold_v1()
        esmfold_model = esmfold_model.eval()
        esmfold_model = esmfold_model.to(device)

        print(f"✅ ESMFold loaded on {device}")
        return esmfold_model
    except Exception as e:
        print(f"❌ Failed to load ESMFold: {e}")
        return None

def load_alphafold():
    """Initialize AlphaFold2 (ColabFold)"""
    global alphafold_runner

    if alphafold_runner is not None:
        return alphafold_runner

    try:
        print("Initializing AlphaFold2 (first time only)...")
        from colabfold.batch import get_queries, run
        from colabfold.download import default_data_dir

        # Download parameters if needed
        import subprocess
        params_dir = "/content/alphafold_params"
        if not os.path.exists(params_dir):
            print("Downloading AlphaFold parameters (~2GB, 3-5 minutes)...")
            os.makedirs(params_dir, exist_ok=True)
            subprocess.run([
                "python", "-m", "colabfold.download",
                "--model-type", "alphafold2_ptm",
                params_dir
            ], check=False)

        alphafold_runner = {
            'params_dir': params_dir,
            'initialized': True
        }

        print("✅ AlphaFold initialized")
        return alphafold_runner
    except Exception as e:
        print(f"❌ Failed to initialize AlphaFold: {e}")
        return None

print("✅ Model loaders ready!")

✅ Model loaders ready!


In [15]:
# Global variables for models
esmfold_model = None
alphafold_runner = None

def load_esmfold():
    """Load ESMFold model"""
    global esmfold_model

    if esmfold_model is not None:
        return esmfold_model

    try:
        print("Loading ESMFold model (first time only, ~2-3 minutes)...")
        import esm

        esmfold_model = esm.pretrained.esmfold_v1()
        esmfold_model = esmfold_model.eval()
        esmfold_model = esmfold_model.to(device)

        print(f"✅ ESMFold loaded on {device}")
        return esmfold_model
    except Exception as e:
        print(f"❌ Failed to load ESMFold: {e}")
        return None

def load_alphafold():
    """Initialize AlphaFold2 (ColabFold)"""
    global alphafold_runner

    if alphafold_runner is not None:
        return alphafold_runner

    try:
        print("Initializing AlphaFold2 (first time only)...")
        from colabfold.batch import get_queries, run
        from colabfold.download import default_data_dir

        # Download parameters if needed
        import subprocess
        params_dir = "/content/alphafold_params"
        if not os.path.exists(params_dir):
            print("Downloading AlphaFold parameters (~2GB, 3-5 minutes)...")
            os.makedirs(params_dir, exist_ok=True)
            subprocess.run([
                "python", "-m", "colabfold.download",
                "--model-type", "alphafold2_ptm",
                params_dir
            ], check=False)

        alphafold_runner = {
            'params_dir': params_dir,
            'initialized': True
        }

        print("✅ AlphaFold initialized")
        return alphafold_runner
    except Exception as e:
        print(f"❌ Failed to initialize AlphaFold: {e}")
        return None

print("✅ Model loaders ready!")

✅ Model loaders ready!


In [16]:
def predict_with_esmfold(sequence, name="protein"):
    """Predict structure using ESMFold"""
    try:
        model = load_esmfold()
        if model is None:
            return None, "ESMFold failed to load"

        print(f"Predicting with ESMFold ({len(sequence)} residues)...")

        with torch.no_grad():
            output = model.infer_pdb(sequence)

        # Save PDB
        pdb_path = f"/content/outputs/{name}_esmfold.pdb"
        with open(pdb_path, 'w') as f:
            f.write(output)

        return pdb_path, None

    except Exception as e:
        return None, f"ESMFold prediction failed: {str(e)}"

def predict_with_alphafold(sequence, name="protein"):
    """Predict structure using AlphaFold2 (ColabFold)"""
    try:
        runner = load_alphafold()
        if runner is None:
            return None, "AlphaFold failed to initialize"

        print(f"Predicting with AlphaFold2 ({len(sequence)} residues)...")

        from colabfold.batch import get_queries, run

        # Create input file
        input_dir = "/content/temp"
        output_dir = "/content/outputs/alphafold_output"
        os.makedirs(output_dir, exist_ok=True)

        fasta_path = f"{input_dir}/{name}.fasta"
        with open(fasta_path, 'w') as f:
            f.write(f">{name}\n{sequence}\n")

        # Run prediction
        queries, is_complex = get_queries(fasta_path)

        run(
            queries=queries,
            result_dir=output_dir,
            use_templates=False,
            num_relax=0,
            num_models=1,
            num_recycles=3,
            model_type="alphafold2_ptm",
            num_seeds=1,
            use_gpu_relax=False,
            keep_existing_results=False,
            rank_by="auto",
            data_dir=runner['params_dir'],
            stop_at_score=100,
        )

        # Find output PDB
        pdb_files = list(Path(output_dir).glob("*_unrelaxed_rank_001*.pdb"))
        if not pdb_files:
            pdb_files = list(Path(output_dir).glob("*.pdb"))

        if pdb_files:
            pdb_path = str(pdb_files[0])
            # Copy to main output
            final_path = f"/content/outputs/{name}_alphafold.pdb"
            import shutil
            shutil.copy(pdb_path, final_path)
            return final_path, None
        else:
            return None, "No PDB output found"

    except Exception as e:
        import traceback
        return None, f"AlphaFold prediction failed: {str(e)}\n{traceback.format_exc()}"

def predict_structure(sequence, model_choice, name="protein"):
    """Main prediction function with model selection"""
    if model_choice == "ESMFold":
        return predict_with_esmfold(sequence, name)
    elif model_choice == "AlphaFold2":
        return predict_with_alphafold(sequence, name)
    else:
        return None, "Invalid model choice"

print("✅ Prediction functions ready!")

✅ Prediction functions ready!


In [17]:
try:
    from openmm import *
    from openmm.app import *
    from openmm.unit import *
    from pdbfixer import PDBFixer
    import mdtraj as md
    MD_AVAILABLE = True
    print("✅ MD simulation available")
except ImportError:
    MD_AVAILABLE = False
    print("⚠️  MD simulation not available")

def run_md_simulation(pdb_path, steps=1000, temperature=300):
    """Run molecular dynamics simulation"""
    if not MD_AVAILABLE:
        return None, None, "MD simulation not available"

    try:
        print(f"Running MD simulation ({steps} steps)...")

        # Fix PDB
        fixer = PDBFixer(filename=pdb_path)
        fixer.findMissingResidues()
        fixer.findMissingAtoms()
        fixer.addMissingAtoms()
        fixer.addMissingHydrogens(7.0)

        # Setup system
        forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
        modeller = Modeller(fixer.topology, fixer.positions)
        modeller.addSolvent(forcefield, model='tip3p', padding=1.0*nanometer)

        system = forcefield.createSystem(
            modeller.topology,
            nonbondedMethod=PME,
            nonbondedCutoff=1.0*nanometer,
            constraints=HBonds
        )

        # Integrator
        integrator = LangevinMiddleIntegrator(
            temperature*kelvin,
            1.0/picosecond,
            0.004*picoseconds
        )

        # Simulation
        simulation = Simulation(modeller.topology, system, integrator)
        simulation.context.setPositions(modeller.positions)

        # Minimize
        print("Energy minimization...")
        simulation.minimizeEnergy(maxIterations=100)

        # Run simulation
        traj_path = pdb_path.replace('.pdb', '_md.pdb')
        simulation.reporters.append(PDBReporter(traj_path, max(steps//10, 1)))

        print(f"Running {steps} steps...")
        simulation.step(steps)

        # Calculate RMSD
        traj = md.load(traj_path)
        rmsd = md.rmsd(traj, traj, 0) * 10  # Convert to Angstroms

        print("✅ MD simulation complete")
        return traj_path, rmsd, None

    except Exception as e:
        import traceback
        return None, None, f"MD simulation failed: {str(e)}\n{traceback.format_exc()}"

print("✅ MD simulation functions ready!")

✅ MD simulation available
✅ MD simulation functions ready!


In [18]:
try:
    from vina import Vina
    from rdkit import Chem
    from rdkit.Chem import AllChem
    from meeko import MoleculePreparation
    DOCKING_AVAILABLE = True
    print("✅ Docking available")
except ImportError:
    DOCKING_AVAILABLE = False
    print("⚠️  Docking not available")

def prepare_ligand(smiles, output_path):
    """Prepare ligand from SMILES"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES string")

        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, randomSeed=42)
        AllChem.MMFFOptimizeMolecule(mol)

        # Save as PDB
        pdb_path = output_path.replace('.pdbqt', '.pdb')
        Chem.MolToPDBFile(mol, pdb_path)

        # Convert to PDBQT
        preparator = MoleculePreparation()
        preparator.prepare(mol)

        with open(output_path, 'w') as f:
            f.write(preparator.write_pdbqt_string())

        return output_path
    except Exception as e:
        raise Exception(f"Ligand preparation failed: {e}")

def calculate_binding_site(pdb_path):
    """Calculate approximate binding site center"""
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_path)

    coords = []
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    coords.append(atom.coord)

    coords = np.array(coords)
    center = coords.mean(axis=0)

    return center[0], center[1], center[2]

def run_docking(receptor_pdb, ligand_smiles):
    """Run molecular docking"""
    if not DOCKING_AVAILABLE:
        return None, None, "Docking not available"

    try:
        print("Preparing docking...")

        # Prepare ligand
        ligand_pdbqt = "/content/temp/ligand.pdbqt"
        prepare_ligand(ligand_smiles, ligand_pdbqt)

        # Calculate binding site
        center_x, center_y, center_z = calculate_binding_site(receptor_pdb)
        print(f"Binding site center: ({center_x:.2f}, {center_y:.2f}, {center_z:.2f})")

        # Setup Vina
        v = Vina(sf_name='vina', verbosity=0)
        v.set_receptor(receptor_pdb)
        v.set_ligand_from_file(ligand_pdbqt)

        # Set search space
        v.compute_vina_maps(
            center=[center_x, center_y, center_z],
            box_size=[25, 25, 25]
        )

        # Dock
        print("Running docking...")
        v.dock(exhaustiveness=8, n_poses=5)

        # Get results
        energies = v.energies(n_poses=5)

        # Save best pose
        output_path = "/content/outputs/docked_complex.pdbqt"
        v.write_poses(output_path, n_poses=1, overwrite=True)

        print(f"✅ Docking complete! Best score: {energies[0][0]:.2f} kcal/mol")
        return output_path, energies, None

    except Exception as e:
        import traceback
        return None, None, f"Docking failed: {str(e)}\n{traceback.format_exc()}"

print("✅ Docking functions ready!")

⚠️  Docking not available
✅ Docking functions ready!


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [19]:
def visualize_structure(pdb_path, style='cartoon', color='spectrum', width=800, height=600):
    """Create 3D visualization"""
    try:
        with open(pdb_path, 'r') as f:
            pdb_data = f.read()

        view = py3Dmol.view(width=width, height=height)
        view.addModel(pdb_data, 'pdb')

        if style == 'cartoon':
            view.setStyle({'cartoon': {'color': color}})
        elif style == 'sphere':
            view.setStyle({'sphere': {'colorscheme': color}})
        elif style == 'stick':
            view.setStyle({'stick': {'colorscheme': color}})

        view.setBackgroundColor('white')
        view.zoomTo()

        return view._make_html()
    except Exception as e:
        return f"<p style='color:red'>Visualization error: {str(e)}</p>"

def plot_rmsd(rmsd_values):
    """Plot RMSD"""
    if rmsd_values is None:
        return None

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=rmsd_values,
        mode='lines',
        line=dict(color='#2E86AB', width=2)
    ))

    fig.update_layout(
        title='RMSD over Time',
        xaxis_title='Frame',
        yaxis_title='RMSD (Å)',
        template='plotly_white',
        height=400
    )

    return fig

def plot_binding_energies(energies):
    """Plot binding energies"""
    if energies is None:
        return None

    poses = list(range(1, len(energies) + 1))
    scores = [e[0] for e in energies]

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=poses,
        y=scores,
        marker_color='#06A77D',
        text=[f"{s:.2f}" for s in scores],
        textposition='outside'
    ))

    fig.update_layout(
        title='Binding Affinity by Pose',
        xaxis_title='Pose Rank',
        yaxis_title='Binding Energy (kcal/mol)',
        template='plotly_white',
        height=400
    )

    return fig

print("✅ Visualization functions ready!")

✅ Visualization functions ready!


In [20]:
def validate_sequence(sequence):
    """Validate amino acid sequence"""
    sequence = sequence.upper().strip().replace(' ', '').replace('\n', '')
    valid_aa = set('ACDEFGHIKLMNPQRSTVWY')

    if not sequence:
        return None, "Empty sequence"

    if not all(aa in valid_aa for aa in sequence):
        invalid = set(sequence) - valid_aa
        return None, f"Invalid amino acids: {invalid}"

    if len(sequence) > 600:
        return None, "Sequence too long (max 600 residues)"

    if len(sequence) < 10:
        return None, "Sequence too short (min 10 residues)"

    return sequence, None

def process_prediction(sequence, model_choice, run_md, md_steps):
    """Main processing function"""
    status_messages = []

    # Validate
    sequence, error = validate_sequence(sequence)
    if error:
        return None, None, None, f"❌ {error}", None

    status_messages.append(f"✅ Sequence validated ({len(sequence)} residues)")

    # Predict structure
    status_messages.append(f"🔬 Starting {model_choice} prediction...")
    pdb_path, error = predict_structure(sequence, model_choice)

    if error:
        return None, None, None, "\n".join(status_messages) + f"\n❌ {error}", None

    status_messages.append(f"✅ Structure predicted successfully")

    # Visualize
    html_view = visualize_structure(pdb_path)

    # MD simulation
    rmsd_plot = None
    if run_md and MD_AVAILABLE:
        status_messages.append(f"🧪 Running MD simulation ({md_steps} steps)...")
        traj_path, rmsd, error = run_md_simulation(pdb_path, steps=md_steps)

        if error:
            status_messages.append(f"⚠️ MD simulation failed: {error}")
        else:
            rmsd_plot = plot_rmsd(rmsd)
            status_messages.append(f"✅ MD simulation complete (avg RMSD: {np.mean(rmsd):.2f} Å)")

    final_status = "\n".join(status_messages)

    return html_view, rmsd_plot, final_status, pdb_path, pdb_path

def process_docking_task(pdb_file, ligand_smiles):
    """Process docking"""
    if pdb_file is None:
        return None, None, "❌ Please predict structure first or upload PDB file"

    if not DOCKING_AVAILABLE:
        return None, None, "❌ Docking not available"

    # Get PDB path
    if isinstance(pdb_file, str):
        pdb_path = pdb_file
    else:
        pdb_path = pdb_file.name

    # Run docking
    docked_path, energies, error = run_docking(pdb_path, ligand_smiles)

    if error:
        return None, None, f"❌ {error}"

    # Visualize
    html_view = visualize_structure(pdb_path)
    energy_plot = plot_binding_energies(energies)

    best_energy = energies[0][0]
    status = f"✅ Docking complete!\nBest binding affinity: {best_energy:.2f} kcal/mol\n\n"
    status += "Top 5 poses:\n"
    for i, (energy, lb, ub) in enumerate(energies[:5], 1):
        status += f"Pose {i}: {energy:.2f} kcal/mol\n"

    return html_view, energy_plot, status

print("✅ Processing functions ready!")

✅ Processing functions ready!


In [ ]:
import gradio as gr

# Create Gradio Interface
# Reverting theme back to Blocks() because launch() does not yet support it in Gradio 4.x
with gr.Blocks(title="FoldSmith Copilot", theme=gr.themes.Soft()) as app:

    gr.Markdown("""
    # 🧬 FoldSmith Copilot
    ### AI-Powered Protein Structure Prediction, MD Simulation & Molecular Docking

    **Choose between ESMFold (fast) or AlphaFold2 (accurate) for structure prediction**
    """)

    pdb_state = gr.State()

    with gr.Tabs():
        # Tab 1: Structure Prediction
        with gr.Tab("🔬 Structure Prediction"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Input")

                    model_choice = gr.Radio(
                        choices=["ESMFold", "AlphaFold2"],
                        value="ESMFold",
                        label="Select Prediction Model",
                        info="ESMFold: Faster (~10-30s). AlphaFold2: More accurate (~2-5min)"
                    )

                    sequence_input = gr.Textbox(
                        label="Amino Acid Sequence",
                        placeholder="Enter sequence (10-600 residues)\nExample: MKFLKFSLLTAVLLSVVFAFSSCG...",
                        lines=6
                    )

                    with gr.Accordion("⚙️ Advanced Options", open=False):
                        run_md_check = gr.Checkbox(
                            label="Run MD Simulation",
                            value=False,
                            info="Simulate protein dynamics (adds 2-5 minutes)"
                        )
                        md_steps_slider = gr.Slider(
                            minimum=500,
                            maximum=5000,
                            value=1000,
                            step=500,
                            label="MD Steps"
                        )

                    predict_btn = gr.Button(
                        "🚀 Predict Structure",
                        variant="primary",
                        size="lg"
                    )

                    status_box = gr.Textbox(
                        label="Status",
                        lines=8,
                        interactive=False
                    )

                    pdb_download = gr.File(label="📥 Download PDB File")

                with gr.Column(scale=2):
                    gr.Markdown("### 3D Structure Visualization")
                    structure_viewer = gr.HTML(label="3D Viewer")

                    with gr.Accordion("📊 MD Simulation Results", open=True):
                        rmsd_plot_output = gr.Plot(label="RMSD Plot")

            # Examples
            gr.Examples(
                examples=[
                    ["MKFLKFSLLTAVLLSVVFAFSSCGDDDDTGYLPPSQAIQDLLKRMKVERGQPINVWCG", "ESMFold"],
                    ["FVNQHLCGSHLVEALYLVCGERGFFYTPKA", "ESMFold"],
                    ["MKTFIFLALLGAAVAFPVDDDDKIVGGYTCAANSIPYQVSLNSGSHFCGG", "AlphaFold2"],
                ],
                inputs=[sequence_input, model_choice]
            )

        # Tab 2: Molecular Docking
        with gr.Tab("🧪 Molecular Docking"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Docking Setup")

                    pdb_upload = gr.File(
                        label="Upload Protein PDB (optional - uses predicted structure)",
                        file_types=[".pdb"]
                    )

                    ligand_smiles_input = gr.Textbox(
                        label="Ligand SMILES String",
                        placeholder="Example: CC(=O)OC1=CC=CC=C1C(=O)O (Aspirin)",
                        value="CC(=O)OC1=CC=CC=C1C(=O)O",
                        lines=3
                    )

                    dock_btn = gr.Button(
                        "🎯 Run Docking",
                        variant="primary",
                        size="lg"
                    )

                    dock_status = gr.Textbox(
                        label="Docking Results",
                        lines=10,
                        interactive=False
                    )

                with gr.Column(scale=2):
                    gr.Markdown("### Docking Results")
                    docked_viewer = gr.HTML(label="Docked Complex")
                    binding_plot = gr.Plot(label="Binding Affinity")

            gr.Examples(
                examples=[
                    ["CC(=O)OC1=CC=CC=C1C(=O)O"],
                    ["CN1C=NC2=C1C(=O)N(C(=O)N2C)C"],
                    ["CC(C)CC1=CC=C(C=C1)C(C)C(=O)O"],
                ],
                inputs=[ligand_smiles_input]
            )

    def safe_process_prediction(*args):
        """Helper to ensure error messages don't crash the gr.File component"""
        res = list(process_prediction(*args))
        # Ensure errors are directed to status box, not file downloader
        if res[3] and isinstance(res[3], str) and ('❌' in res[3] or 'failed' in res[3].lower()):
            res[2] = res[3]
            res[3] = None
            res[4] = None
        return tuple(res)

    # Event handlers
    predict_btn.click(
        fn=safe_process_prediction,
        inputs=[sequence_input, model_choice, run_md_check, md_steps_slider],
        outputs=[structure_viewer, rmsd_plot_output, status_box, pdb_download, pdb_state]
    )

    dock_btn.click(
        fn=lambda pdb_file, pdb_state_val, smiles: process_docking_task(
            pdb_file if pdb_file is not None else pdb_state_val, smiles
        ),
        inputs=[pdb_upload, pdb_state, ligand_smiles_input],
        outputs=[docked_viewer, binding_plot, dock_status]
    )

print("✅ Gradio interface created!")
app.launch(share=True, debug=True)

/tmp/ipython-input-2101880483.py:5: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="FoldSmith Copilot", theme=gr.themes.Soft()) as app:
/usr/local/lib/python3.12/dist-packages/gradio/component_meta.py:189: DeprecationWarning: The default value of 'padding' in gr.HTML will be changed from True to False in Gradio 6.0. You will need to explicitly set padding=True if you want to keep the padding.
  ).copy()
/usr/local/lib/python3.12/dist-packages/gradio/helpers.py:445: DeprecationWarning: The 'show_api' parameter in event listeners will be removed in Gradio 6.0. You will need to use the 'api_visibility' parameter instead. To replicate show_api=False, in Gradio 6.0, use api_visibility='undocumented'.
  tempdir = os.environ.get("GRADIO_TEMP_DIR") or str(
/usr/local/lib/python3.12/dist-packages/gradio/helpers.py:85: DeprecationWarning: The 'show_api' parameter i

✅ Gradio interface created!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


* Running on public URL: https://e09706ea7875a0a9e2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
